# Experiments by changing the threshold of reliable positives (AADB dataset)

In [1]:
import sys
sys.path.append("..")

In [18]:
import numpy as np
import pandas as pd
import traceback
import os

from pu.feature_extractors.extractors import ViTExtractor, AutoencoderExtractor
from pu.data.loaders import CSVLoader, SingleCSVLoader, SingleCSVWithTestLoader, FullCSVLoader
from pu.data.pu_builder import build_pu_data, pn_test_split

from pu.algorithms.pu_algorithms import IterativeClassifierAlgorithm, ProbTagging
from pu.algorithms.negative_detectors import NaiveDetector, KNNDetector
from pu.algorithms.stop_criterion import StopOnMetricDrop, NonStop

from sklearn import svm
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score

## AADB dataset for positive and unlabeled examples

In [27]:
def quantile_experiment(extractor, quantile):

    aadb_train_loader = FullCSVLoader(
        '/srv/aadb/train.csv',
        'path',
        '/srv/aadb'
    )

    aadb_val_loader = FullCSVLoader(
        '/srv/aadb/validation.csv',
        'path',
        '/srv/aadb'
    )

    aadb_test_loader = FullCSVLoader(
        '/srv/aadb/testnew.csv',
        'path',
        '/srv/aadb'
    )

    train_extractor = ViTExtractor('quantile_experiment_aadb_train', extractor)
    val_extractor = ViTExtractor('quantile_experiment_aadb_val', extractor)
    test_extractor = ViTExtractor('quantile_experiment_aadb_test', extractor)
    
    aadb_train = aadb_train_loader.load_data()
    aadb_val = aadb_val_loader.load_data()
    aadb_test = aadb_test_loader.load_data()
    
    features_train = train_extractor.extract_features(aadb_train['path'])
    features_val = val_extractor.extract_features(aadb_val['path'])
    features_test = test_extractor.extract_features(aadb_test['path'])
    
    aadb_train = pd.concat([aadb_train.drop(columns=['path']), features_train.drop(columns=['id'])], axis=1)
    aadb_val = pd.concat([aadb_val.drop(columns=['path']), features_val.drop(columns=['id'])], axis=1)
    aadb_test = pd.concat([aadb_test.drop(columns=['path']), features_test.drop(columns=['id'])], axis=1)
    
    X_train, _, _, y_train, _, _ = build_pu_data(
        aadb_train,
        frac=1.0,
        move_to_unlabeled_frac=0.5,
        val_split=0,
        val_split_positive='same',
        reliable_positive_fn=lambda row, df: row['label'] > quantile,
        positive_fn=lambda row, df: row['label'] >= 0.5,
        test_frac=0,
        random_state=1234
    )

    _, X_val, _, _, y_val, _ = build_pu_data(
        aadb_val,
        frac=1.0,
        move_to_unlabeled_frac=0.5,
        val_split=1.0,
        val_split_positive='same',
        reliable_positive_fn=lambda row, df: row['label'] > quantile,
        positive_fn=lambda row, df: row['label'] >= 0.5,
        test_frac=0,
        random_state=1234
    )

    _, _, X_test, y_test = pn_test_split(
        aadb_test, 
        lambda row, df: row['label'] > quantile, 
        lambda row, df: row['label'] >= 0.5, 
        1.0, 
        random_state=1234
    )

    iterative_cls = IterativeClassifierAlgorithm(
        negative_detector=KNNDetector(frac=0.1, k=20),
        stop_criterion=NonStop('aul'),
        classifier_class=LogisticRegression,
        max_iterations=20,
        verbose=True,
        classifier_kwargs={'max_iter':10000, 'solver':'saga', 'n_jobs':-1, 'random_state':1234}
    )
    
    iterative_cls.fit(X_train, y_train, X_val, y_val)
    print(f'Evolution of f1 score: {iterative_cls.validation_results}')

    bal_acc = balanced_accuracy_score(y_test, iterative_cls.predict(X_test))
    acc = accuracy_score(y_test, iterative_cls.predict(X_test))
    f1 = f1_score(y_test, iterative_cls.predict(X_test))

    return bal_acc, acc, f1

In [28]:
extractors = ['clip-ViT-B-32', 'clip-ViT-B-16', 'clip-ViT-L-14']

# Quantiles [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
quantile_quantities = [
    0.5,
    0.55,
    0.55,
    0.6,
    0.6,
    0.65,
    0.65,
    0.7,
    0.75,
    0.8,
    0.9
]

extractor_col, quantile_col, bal_acc_col, acc_col, f1_col = [], [], [], [], []

for quantile in quantile_quantities:
    for extractor in extractors:
        try:
            bal_acc, acc, f1 = quantile_experiment(extractor, quantile)
            extractor_col.append(extractor)
            quantile_col.append(quantile)
            bal_acc_col.append(bal_acc)
            acc_col.append(acc)
            f1_col.append(f1)

        except Exception as e:
            print(f'Fail at {quantile}, {extractor}')
            print(traceback.format_exc())

df = pd.DataFrame.from_dict({
    'extractor': extractor_col,
    'quantile': quantile_col,
    'balanced_accuracy': bal_acc_col,
    'accuracy': acc_col,
    'f1': f1_col
})

df.to_csv('quantile_threshold_vit_results.csv')

Total amount: 8458
Validation amount: 0
Val_positive amount: 0
Val_unlabeled amount: 0
Train amount: 8458
Known_positive amount: 2010
Unlabeled_amount: 6448
Size of positive paths: 4020
Size of unlabeled paths: 4438
Total amount: 500
Validation amount: 500
Val_positive amount: 244
Val_unlabeled amount: 256
Train amount: 0
Known_positive amount: 0
Unlabeled_amount: 0
Size of positive paths: 244
Size of unlabeled paths: 256
Iteration #0
Number of negatives: 644
Number of positives: 2010
Finished training classifier
Validation metric for iteration 0: 1.492429652470103
Moving 58 from unlabeled to negative and from unlabeled to positive
Iteration #1
Number of negatives: 702
Number of positives: 2068
Finished training classifier
Validation metric for iteration 1: 1.4810286980373535
Moving 56 from unlabeled to negative and from unlabeled to positive
Iteration #2
Number of negatives: 758
Number of positives: 2124
Finished training classifier
Validation metric for iteration 2: 1.470325876691896

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Finished training classifier
Validation metric for iteration 19: 1.4253815857689276
Moving 39 from unlabeled to negative and from unlabeled to positive
Evolution of f1 score: [1.471033543697146, 1.4750507635987389, 1.4774529487437285, 1.4644321679071137, 1.4644321679071137, 1.4619941743873917, 1.4646822659369332, 1.4690926127828103, 1.4638345525824632, 1.4626062260282926, 1.4424883346533028, 1.4592721563730429, 1.4594295521698921, 1.4428226810038849, 1.4423504829081661, 1.4395901447506654, 1.4426466302258383, 1.4371411917739665, 1.4291783945547951, 1.4253815857689276]


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Total amount: 8458
Validation amount: 0
Val_positive amount: 0
Val_unlabeled amount: 0
Train amount: 8458
Known_positive amount: 2010
Unlabeled_amount: 6448
Size of positive paths: 4020
Size of unlabeled paths: 4438
Total amount: 500
Validation amount: 500
Val_positive amount: 244
Val_unlabeled amount: 256
Train amount: 0
Known_positive amount: 0
Unlabeled_amount: 0
Size of positive paths: 244
Size of unlabeled paths: 256
Iteration #0
Number of negatives: 644
Number of positives: 2010
Finished training classifier
Validation metric for iteration 0: 1.5697866535291116
Moving 58 from unlabeled to negative and from unlabeled to positive
Iteration #1
Number of negatives: 702
Number of positives: 2068
Finished training classifier
Validation metric for iteration 1: 1.4532151374060098
Moving 56 from unlabeled to negative and from unlabeled to positive
Iteration #2
Number of negatives: 758
Number of positives: 2124
Finished training classifier
Validation metric for iteration 2: 1.45527036591629

...

Total amount: 8458
Validation amount: 0
Val_positive amount: 0
Val_unlabeled amount: 0
Train amount: 8458
Known_positive amount: 169
Unlabeled_amount: 8288
Size of positive paths: 339
Size of unlabeled paths: 8119
Total amount: 500
Validation amount: 500
Val_positive amount: 21
Val_unlabeled amount: 479
Train amount: 0
Known_positive amount: 0
Unlabeled_amount: 0
Size of positive paths: 21
Size of unlabeled paths: 479
Iteration #0
Number of negatives: 828
Number of positives: 170
Finished training classifier
Validation metric for iteration 0: 1.1416492382033563
Moving 74 from unlabeled to negative and from unlabeled to positive
Iteration #1
Number of negatives: 902
Number of positives: 244
Finished training classifier
Validation metric for iteration 1: 1.1955825478051991
Moving 73 from unlabeled to negative and from unlabeled to positive
Iteration #2
Number of negatives: 975
Number of positives: 317
Finished training classifier
Validation metric for iteration 2: 1.210865005940337
Movin

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Finished training classifier
Validation metric for iteration 19: 1.8800005698598847
Moving 51 from unlabeled to negative and from unlabeled to positive
Evolution of f1 score: [2.170178290089289, 2.0715893007345962, 1.996170430456579, 1.9964366357508934, 1.9391057433962855, 1.4478502404807285, 1.4178722185027064, 1.4178722185027064, 1.3853536943257911, 1.4403741941002544, 1.4178722185027064, 1.4071848103845548, 1.3748273785363174, 1.9728634469917798, 1.9265843851366027, 1.8801788229793142, 1.4028369842975983, 1.3748273785363174, 1.8716467836784287, 1.8800005698598847]


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Total amount: 8458
Validation amount: 0
Val_positive amount: 0
Val_unlabeled amount: 0
Train amount: 8458
Known_positive amount: 23
Unlabeled_amount: 8435
Size of positive paths: 46
Size of unlabeled paths: 8412
Total amount: 500
Validation amount: 500
Val_positive amount: 5
Val_unlabeled amount: 495
Train amount: 0
Known_positive amount: 0
Unlabeled_amount: 0
Size of positive paths: 5
Size of unlabeled paths: 495
Iteration #0
Number of negatives: 843
Number of positives: 23
Finished training classifier
Validation metric for iteration 0: 2.420548569301628
Moving 75 from unlabeled to negative and from unlabeled to positive
Iteration #1
Number of negatives: 918
Number of positives: 98
Finished training classifier
Validation metric for iteration 1: 2.428234742242059
Moving 74 from unlabeled to negative and from unlabeled to positive
Iteration #2
Number of negatives: 992
Number of positives: 172
Finished training classifier
Validation metric for iteration 2: 2.289700048568683
Moving 72 fro